# Random Walks in Static Networks

[Run notebook in Google Colab](https://colab.research.google.com/github/pathpy/pathpy/blob/master/doc/tutorial/random_walks.ipynb)  
[Download notebook](https://github.com/pathpy/pathpy/raw/master/doc/tutorial/random_walks.ipynb)

In [ ]:
pip install git+git://github.com/pathpy/pathpy.git

In [ ]:
import pathpy as pp
from pprint import pprint

We first generate a directed network with weighted edges:

In [ ]:
n = pp.Network(directed=True)
n.add_edge('a', 'b', weight=1, uid='a-b')
n.add_edge('b', 'c', weight=1, uid='b-c')
n.add_edge('c', 'a', weight=2, uid='c-a')
n.add_edge('c', 'd', weight=1, uid='c-d')
n.add_edge('d', 'a', weight=1, uid='d-a')
n.plot()

To initialize a random walk process, we can generate a `RandomWalk` instance on the network. Specifying the `weight` attribute will lead to a biased random walk, where the transition probabilities are weighted by the respective numerical property:

In [ ]:
rw = pp.processes.RandomWalk(n, weight='weight')

We can inspect the transition matrix of the random walk process as follows. Using the function `matrix_pd` will return the matrix as a (nicely formatted) pandas DataFrame, while `matrix` returns the internal sparse matrix representation.

In [ ]:
print(rw.transition_matrix_pd())

We can use the method `generate_walk` of the random walk instance to generate a random walk with a given length. If we specify `start_node`, the walk will start from the given node object. The method returns a `Path` object.

In [ ]:
for time, updated_nodes in rw.simulation_run(steps=10, seed='a'):
    print('time = {0}, node = {1}'.format(time, updated_nodes[0]))

In [ ]:
data = rw.run_experiment(steps=10, runs=['a', 'b'])
print(data)

In [ ]:
p = rw.get_path(data, 0)
print([v for v in p.nodes])

In [ ]:
pc = rw.get_paths(data, [0,1])
print(pc)

In [ ]:
rw.plot(data)

If we omit the `start_node` argument, a random node will be chosen as start node.

In [ ]:
data = rw.run_experiment(steps=200, runs=2)
rw.plot(data)

We can generate a `PathCollection` that contains a given number of random walks with a given length specified by `steps_per_walk`. If the `start_nodes` argument is a numeric, the given number of walks will be generated, each walk starting from a random node.

In [ ]:
pc = rw.get_paths(rw.run_experiment(steps=10, runs=10))
for p in pc:
    print( tuple([v.uid for v in p.nodes]))

If `start_nodes` is a list of `Node` objects, a single random walk will be generated for each start node. To generate a single random walk starting in each node of the network, we can write:

In [ ]:
pc = rw.get_paths(rw.run_experiment(steps=10, runs=n.nodes.uids))
for p in pc:
    print(tuple([ v.uid for v in p.nodes ]))

We can also use an iterator interface to iteratively perform random walk steps. I neach step, we can assess the current state of the random walk at time t.

In [ ]:
for time, _ in rw.simulation_run(10, seed='a'):
    print('Current node = {0}'.format(rw.current_node))
    print('Current time = {0}'.format(rw.time))
    print(rw.visitation_frequencies)    
    print(rw.total_variation_distance)

The visitation probabilities after t steps for a given start node can be computed using the `visitation_probabilities` function.

In [ ]:
rw.visitation_probabilities(0, seed='a')

In [ ]:
rw.visitation_probabilities(1, seed='a')

In [ ]:
rw.visitation_probabilities(100, seed='a')

The stationary visitation probabilities for $t \rightarrow \infty$ can be computed as follows:

In [ ]:
rw.stationary_state()